<a href="https://colab.research.google.com/github/mikashaw/ML-Projects/blob/main/MusicNet/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Flatten, Dense, Reshape, Conv2DTranspose, Activation
from tensorflow.keras import backend as K
import numpy as np

In [17]:
class Autoencoder:
  """
  Autoencoder represents a Deep Convolutional autoencoder architecture with mirrored encoder and decoder components
  """

  def __init__(self, 
               input_shape,
               conv_filters,
               conv_kernels,
               conv_strides,
               latent_space_dim):
    self.input_shape = input_shape # [28, 28, 1]
    self.conv_filters = conv_filters # [2,4,8]
    self.conv_kernels = conv_kernels # [3,5,3]
    self.conv_strides = conv_strides # [1,2,2]
    self.latent_space_dim = latent_space_dim # 2
    self._shape_before_bottleneck = None

    self.encoder = None
    self.decoder = None
    self.model = None

    self._num_conv_layers = len(conv_filters)

    self._build()

  def summary(self):
    self.encoder.summary()
    self.decoder.summary()

  def _build(self):
    self._build_encoder()
    self._build_decoder()
    #self._build_autoencoder()

  def _build_decoder(self):
    decoder_input = self._add_decoder_input()
    dense_layer = self._add_dense_layer(decoder_input)
    reshape_layer = self._add_reshape_layer(dense_layer)
    conv_transpose_layers = self._add_conv_transpose_layers(reshape_layer)
    decoder_output = self._add_decoder_output(conv_transpose_layers)
    self.decoder = Model(decoder_input, decoder_output, name ="decoder")

  def _add_decoder_input(self):
    return Input(shape=self.latent_space_dim, name = "decoder_input")

  def _add_dense_layer(self, decoder_input):
    num_neurons = np.prod(self._shape_before_bottleneck) #[1,2,4] -> 8
    dense_layer = Dense(num_neurons, name = "decoder_dense")(decoder_input)
    return dense_layer

  def _add_reshape_layer(self, dense_layer):
    reshape_layer = Reshape(self._shape_before_bottleneck)(dense_layer)
    return reshape_layer

  def _add_conv_transpose_layers(self, x):
    """Add conv transpose blocks"""
    # loop through all the conv later in reverse order and stop at the first layer
    for layer_index in reversed(range(1, self._num_conv_layers)):
      # [0,1,2] -> [2,1,0]
      x = self._add_conv_transpose_layer(layer_index, x)
    return x

  def _add_conv_transpose_layer(self, layer_index, x):
    layer_num = self._num_conv_layers - layer_index
    conv_transpose_layer = Conv2DTranspose(
        filters = self.conv_filters[layer_index],
        kernel_size = self.conv_kernels[layer_index],
        strides = self.conv_strides[layer_index],
        padding="same",
        name=f"decoder_conv_transpose_layer_{layer_num}"
    )

    x = conv_transpose_layer(x)
    x = ReLU(name=f"decoder_relu_{layer_num}")(x)
    x = BatchNormalization(name=f"decoder_bn_{layer_num}")(x)
    return x

  def _add_decoder_output(self, x):
    conv_transpose_layer = Conv2DTranspose(
        filters = 1,
        kernel_size = self.conv_kernels[0],
        strides = self.conv_strides[0],
        padding = "same",
        name=f"decoder_conv_transpose_layer_{self._num_conv_layers}"
    )
    x = conv_transpose_layer(x)
    output_layer = Activation("sigmoid", name = "sigmoid_layer")(x)
    return output_layer

  def _build_encoder(self):
    encoder_input = self._add_encoder_input()
    conv_layers = self._add_conv_layers(encoder_input)
    bottleneck = self._add_bottleneck(conv_layers)
    self.encoder = Model(encoder_input, bottleneck, name = "encoder")

  def _add_encoder_input(self):
    return Input(shape = self.input_shape, name="encoder_input")

  def _add_conv_layers(self, encoder_input):
    """
    Creates all convolutionals block in encoder
    """
    x = encoder_input
    for layer_index in range(self._num_conv_layers):
      x = self._add_conv_layer(layer_index, x)
    return x

  def _add_conv_layer(self, layer_index, x):
    """
    Adds a convolutional block to a graph of layers, consisting of a conv 2d + ReLU + batch normalization.
    """
    layer_number = layer_index + 1
    conv_layer = Conv2D(
        filters = self.conv_filters[layer_index],
        kernel_size = self.conv_kernels[layer_index],
        strides = self.conv_strides[layer_index],
        padding= "same",
        name=f"encoder_conv_layer_{layer_number}"
    
    )
    x = conv_layer(x)
    x = ReLU(name = f"encoder_relu_{layer_number}")(x)
    x = BatchNormalization(name=f"encoder_bn_{layer_number}")(x)
    return x

  def _add_bottleneck(self, x):
    """Flatten data and add bottle neck (Dense layer)"""
    self._shape_before_bottleneck = K.int_shape(x)[1:] #[2,7,7,1]
    x = Flatten()(x)
    x = Dense(self.latent_space_dim, name="encoder_output")(x)
    return x





In [18]:
autoencoder = Autoencoder(
    input_shape = (28,28,1),
    conv_filters = (32,64,64,64),
    conv_kernels = (3,3,3,3),
    conv_strides = (1,2,2,1),
    latent_space_dim=2

)

In [19]:
autoencoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_layer_1 (Conv2  (None, 28, 28, 32)       320       
 D)                                                              
                                                                 
 encoder_relu_1 (ReLU)       (None, 28, 28, 32)        0         
                                                                 
 encoder_bn_1 (BatchNormaliz  (None, 28, 28, 32)       128       
 ation)                                                          
                                                                 
 encoder_conv_layer_2 (Conv2  (None, 14, 14, 64)       18496     
 D)                                                              
                                                           